In [1]:
import rasterio
import geopandas as gpd
import pandas as pd
import glob
import os
from xrspatial import zonal_stats
# from rasterstats import raster_stats
from geocube.api.core import make_geocube
import rioxarray
from shapely.geometry import box
import numpy as np

In [2]:
vhm = rioxarray.open_rasterio('dados/0-VHM-sao-paulo-city-1m-8bits.tif')
vhm_100 = rioxarray.open_rasterio('dados/0-VHM-sao-paulo-city-1m-8bits-raio-100m.tif')
vhm_1000 = rioxarray.open_rasterio('dados/0-VHM-sao-paulo-city-1m-8bits-raio-1000m.tif')

In [4]:
results = []

for l in glob.glob('dados/lotes/*.zip'):

    print(os.path.splitext(os.path.basename(l))[0])
    gdf_lote = gpd.read_file(f'{l}!{os.path.splitext(os.path.basename(l))[0]}')
    gdf_lote["sqlc"] = gdf_lote.lo_setor + gdf_lote.lo_quadra + gdf_lote.lo_lote + gdf_lote.lo_condomi
    gdf_lote = gdf_lote.loc[gdf_lote.lo_tp_lote == 'F', ["sqlc", "geometry"]]
    gdf_lote = gdf_lote.reset_index().rename(columns = {'index':'indice'})
    # gdf_lote.indice = gdf_lote.indice + 1

    gdf_lote_bb = gpd.GeoDataFrame(
        geometry=[box(*list(gdf_lote.geometry.total_bounds))],
        crs='EPSG:31983'
    )

    clipped_vhm = vhm.rio.clip(gdf_lote_bb.geometry, from_disk=True).sel(band=1).drop("band")
    clipped_vhm_100 = vhm_100.rio.clip(gdf_lote_bb.geometry, from_disk=True).sel(band=1).drop("band")
    clipped_vhm_1000 = vhm_1000.rio.clip(gdf_lote_bb.geometry, from_disk=True).sel(band=1).drop("band")

    # minx, miny, maxx, maxy = list(gdf_lote.geometry.total_bounds)
    # clipped_vhm = vhm.rio.clip_box(minx, miny, maxx, maxy, auto_expand=True).sel(band=1).drop("band")
    # clipped_vhm_100 = vhm_100.rio.clip_box(minx, miny, maxx, maxy, auto_expand=True).sel(band=1).drop("band")
    # clipped_vhm_1000 = vhm_1000.rio.clip_box(minx, miny, maxx, maxy, auto_expand=True).sel(band=1).drop("band")

    # rasterizar cada lote com o SQL
    out_grid = make_geocube(
        vector_data=gdf_lote,
        measurements=["indice"],
        # like=clipped_vhm,
        resolution=(1.0, -1.0), 
        fill=999999.
    )
    
    out_grid["indice"] = out_grid.indice.astype(int)

    min_dim_sizes = np.min(np.stack([np.array(out_grid.indice.shape), np.array(clipped_vhm.shape)]), axis=0)

    zonas = zonal_stats(
        zones=out_grid.indice[:min_dim_sizes[0], :min_dim_sizes[1]],
        values=clipped_vhm[:min_dim_sizes[0], :min_dim_sizes[1]],
        # nodata_values=0,
        # affine=vhm_src.transform,
        stats_funcs=['mean']
    ).drop(columns=['zone']).rename(
        columns={'mean':'media_veg_lote'}
    )
    
    zonas_100 = zonal_stats(
        zones=out_grid.indice[:min_dim_sizes[0], :min_dim_sizes[1]],
        values=clipped_vhm_100[:min_dim_sizes[0], :min_dim_sizes[1]],
        # nodata_values=0,
        # affine=vhm_src.transform,
        stats_funcs=['mean']
    ).drop(columns=['zone']).rename(
        columns={'mean':'media_veg_100m'}
    )

    zonas_1000 = zonal_stats(
        zones=out_grid.indice[:min_dim_sizes[0], :min_dim_sizes[1]],
        values=clipped_vhm_1000[:min_dim_sizes[0], :min_dim_sizes[1]],
        # nodata_values=0,
        # affine=vhm_src.transform,
        stats_funcs=['mean']
    ).drop(columns=['zone']).rename(
        columns={'mean':'media_veg_1000m'}
    )

    result = pd.concat([gdf_lote, zonas, zonas_100, zonas_1000], axis=1)
    results.append(result.drop(['indice', 'geometry'], axis=1))
    # print(os.path.splitext(os.path.basename(l))[0])
    # break

SIRGAS_SHP_LOTES_34_IPIRANGA
SIRGAS_SHP_LOTES_94_VILA_SONIA
SIRGAS_SHP_LOTES_27_CURSINO
SIRGAS_SHP_LOTES_21_CASA_VERDE
SIRGAS_SHP_LOTES_69_SANTA_CECILIA
SIRGAS_SHP_LOTES_03_ANHANGUERA
SIRGAS_SHP_LOTES_58_PEDREIRA
SIRGAS_SHP_LOTES_02_ALTO_DE_PINHEIROS
SIRGAS_SHP_LOTES_08_BELEM
SIRGAS_SHP_LOTES_87_VILA_JACUI
SIRGAS_SHP_LOTES_25_CIDADE_TIRADENTES
SIRGAS_SHP_LOTES_07_BELA_VISTA
SIRGAS_SHP_LOTES_71_SANTO_AMARO
SIRGAS_SHP_LOTES_41_JAGUARE
SIRGAS_SHP_LOTES_84_VILA_CURUCA
SIRGAS_SHP_LOTES_62_PINHEIROS
SIRGAS_SHP_LOTES_19_CAPAO_REDONDO
SIRGAS_SHP_LOTES_32_MOEMA
SIRGAS_SHP_LOTES_29_FREGUESIA_DO_O
SIRGAS_SHP_LOTES_64_PONTE_RASA
SIRGAS_SHP_LOTES_85_VILA_FORMOSA
SIRGAS_SHP_LOTES_13_CACHOEIRINHA
SIRGAS_SHP_LOTES_72_SAO_LUCAS
SIRGAS_SHP_LOTES_47_JOSE_BONIFACIO
SIRGAS_SHP_LOTES_36_ITAIM_PAULISTA
SIRGAS_SHP_LOTES_45_JARDIM_PAULISTA
SIRGAS_SHP_LOTES_55_PARELHEIROS
SIRGAS_SHP_LOTES_43_JARDIM_ANGELA
SIRGAS_SHP_LOTES_95_SAO_DOMINGOS
SIRGAS_SHP_LOTES_46_JARDIM_SAO_LUIS
SIRGAS_SHP_LOTES_86_VILA_GUILHERME
SIR

In [5]:
pd.concat(results).reset_index().to_feather('resultados/dimensoes_vegetacao.feather')

In [9]:
out_grid["indice"].rio.to_raster("tmp/my_rasterized_column.tif")